In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tf_explain.core.grad_cam import GradCAM
from IPython.display import Image, display

# Define paths for the images and indices of interest
IMAGE_PATHS = ['img1.jpg', 'img2.jpg']  # Replace with actual paths
indices = [263, 281]  # Indices of the class predictions
layers_name = ['block5_conv3']  # Layer to visualize

# Loop through each image and process
for i in range(len(IMAGE_PATHS)):
    each_path = IMAGE_PATHS[i]
    index = indices[i]
    
    # Load and preprocess the image
    img = load_img(each_path, target_size=(224, 224))
    img = img_to_array(img)
    img = tf.expand_dims(img, axis=0)  # Add batch dimension
    img = tf.keras.applications.vgg16.preprocess_input(img)  # Preprocess for VGG16
    
    data = (img, None)  # Data format for tf-explain GradCAM
    
    # Define name for saving the result
    name = each_path.split("/")[-1].split(".jpg")[0]
    
    # Initialize Grad-CAM explainer
    explainer = GradCAM()
    model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
    
    # Generate and save Grad-CAM visualization
    grid = explainer.explain(data, model, layers_name[0], index)
    explainer.save(grid, '.', name + '_grad_cam.png')
    
    # Display the Grad-CAM visualization
    display(Image(filename=name + '_grad_cam.png'))


ModuleNotFoundError: No module named 'tf_explain'

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Helper function to preprocess and load an image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)  # For VGG16
    return img

# Helper function to generate Grad-CAM
def generate_gradcam(model, img_array, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the top predicted class or specified class
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Compute gradients of the target class w.r.t. the convolutional outputs
    grads = tape.gradient(class_channel, conv_outputs)

    # Compute the mean of gradients over the spatial dimensions
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply the pooled gradients with the convolutional outputs
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize the heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()

# Helper function to overlay the heatmap on the image
def overlay_heatmap(heatmap, img_path, alpha=0.4):
    img = load_img(img_path)
    img = img_to_array(img)

    heatmap = np.uint8(255 * heatmap)
    heatmap = np.expand_dims(heatmap, axis=-1)
    heatmap = tf.image.resize(heatmap, (img.shape[0], img.shape[1])).numpy()

    heatmap = tf.keras.preprocessing.image.array_to_img(heatmap)
    heatmap = heatmap.resize((img.shape[1], img.shape[0]))

    heatmap = np.array(heatmap)
    overlayed = img * (1 - alpha) + heatmap[..., np.newaxis] * alpha
    return overlayed.astype('uint8')

# Paths and parameters
IMAGE_PATHS = ['img1.jpg', 'img2.jpg']  # Replace with actual paths
indices = [263, 281]  # Class indices for predictions
last_conv_layer_name = "block5_conv3"  # Last convolutional layer in VGG16

# Load pre-trained VGG16 model
model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

# Generate Grad-CAM for each image
for i, image_path in enumerate(IMAGE_PATHS):
    img_array = preprocess_image(image_path)

    # Generate Grad-CAM heatmap
    heatmap = generate_gradcam(model, img_array, last_conv_layer_name, pred_index=indices[i])

    # Overlay the heatmap on the image
    overlayed_image = overlay_heatmap(heatmap, image_path)

    # Save and display the Grad-CAM visualization
    plt.figure()
    plt.imshow(overlayed_image)
    plt.axis('off')
    plt.title(f"Grad-CAM for Class Index: {indices[i]}")
    plt.show()


 61808640/553467096 ━━━━━━━━━━━━━━━━━━━━ 16:10 2us/step

KeyboardInterrupt: 